In [14]:
μ_min1 = Array{Float64}(undef, 2, 2, 2, 2)
π0 = Array{Float64}(undef, 2 ,2 , 2, 2)
β = 0.8
ν = 1.1

1.1

1.1

In [15]:
using Random
Random.seed!(123)

TaskLocalRNG()

TaskLocalRNG()

In [16]:
for i in 1:2
    for j in 1:2
        matrix = rand(2, 2)
        matrix2 = matrix/sum(matrix)
        μ_min1[i, j, :, :] = matrix2
    end
end
μ_min1

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 0.238018  0.277479
 0.255638  0.0421295

[:, :, 2, 1] =
 0.267972  0.206178
 0.144093  0.479114

[:, :, 1, 2] =
 0.40683   0.0236579
 0.232052  0.373495

[:, :, 2, 2] =
 0.0871797  0.492685
 0.368217   0.105261

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 0.238018  0.277479
 0.255638  0.0421295

[:, :, 2, 1] =
 0.267972  0.206178
 0.144093  0.479114

[:, :, 1, 2] =
 0.40683   0.0236579
 0.232052  0.373495

[:, :, 2, 2] =
 0.0871797  0.492685
 0.368217   0.105261

In [17]:
for i in 1:2
    for j in 1:2
        matrix = rand(2, 2)
        matrix2 = matrix/sum(matrix)
        π0[i, j, :, :] = matrix2
    end
end
π0

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 0.31954  0.116465
 0.41648  0.314584

[:, :, 2, 1] =
 0.25454  0.414087
 0.14425  0.484241

[:, :, 1, 2] =
 0.294853   0.370912
 0.0965257  0.139731

[:, :, 2, 2] =
 0.131067  0.0985356
 0.342745  0.0614439

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 0.31954  0.116465
 0.41648  0.314584

[:, :, 2, 1] =
 0.25454  0.414087
 0.14425  0.484241

[:, :, 1, 2] =
 0.294853   0.370912
 0.0965257  0.139731

[:, :, 2, 2] =
 0.131067  0.0985356
 0.342745  0.0614439

In [18]:
κdot = rand(0.95:0.001:1.08, (2, 2, 2, 2))

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 1.026  0.965
 0.99   0.976

[:, :, 2, 1] =
 1.0    0.959
 0.952  1.069

[:, :, 1, 2] =
 1.0    0.999
 1.051  1.064

[:, :, 2, 2] =
 0.984  1.063
 1.068  1.045

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 1.026  0.965
 0.99   0.976

[:, :, 2, 1] =
 1.0    0.959
 0.952  1.069

[:, :, 1, 2] =
 1.0    0.999
 1.051  1.064

[:, :, 2, 2] =
 0.984  1.063
 1.068  1.045

In [19]:
Ȧ = rand(0.98:0.001:1.05, (2, 2))

2×2 Matrix{Float64}:
 0.987  1.037
 1.021  0.995

2×2 Matrix{Float64}:
 0.987  1.037
 1.021  0.995

In [20]:
L_setup_matrix = rand(2, 2)
L0 = L_setup_matrix/sum(L_setup_matrix)

2×2 Matrix{Float64}:
 0.303201  0.23899
 0.132584  0.325225

2×2 Matrix{Float64}:
 0.303201  0.23899
 0.132584  0.325225

In [21]:
w0 = rand(0.5:0.001:1.5, (2, 2))

2×2 Matrix{Float64}:
 1.207  1.379
 1.196  0.917

2×2 Matrix{Float64}:
 1.207  1.379
 1.196  0.917

In [22]:
using Distributions
θ_var = Frechet()
θ_list = rand(θ_var, 2)

2-element Vector{Float64}:
 8.406813510366845
 2.4055503080991634

2-element Vector{Float64}:
 8.406813510366845
 2.4055503080991634

In [23]:
u̇_0t = rand(0.96:0.001:1.04, (2, 2, 2))
# here, u̇_0t is the the initial guess for u̇ in both the time periods, where u̇_0t[:, :, 1] is for the first time period
# and u̇_0t[:, :, 2] is for the second time period.

2×2×2 Array{Float64, 3}:
[:, :, 1] =
 1.007  0.997
 1.005  1.026

[:, :, 2] =
 1.006  1.003
 1.025  1.022

2×2×2 Array{Float64, 3}:
[:, :, 1] =
 1.007  0.997
 1.005  1.026

[:, :, 2] =
 1.006  1.003
 1.025  1.022

In [24]:
ẇ = rand(0.96:0.001:1.04, (2, 2))

2×2 Matrix{Float64}:
 0.965  1.028
 1.035  1.024

2×2 Matrix{Float64}:
 0.965  1.028
 1.035  1.024

In [25]:
μ_0 = Array{Float64}(undef, 2, 2, 2, 2)
L1 = Array{Float64}(undef, 2, 2)
ẇn = Array{Float64}(undef, 2, 2)#This is going to be our updated wage change guess.
Ṗ = Array{Float64}(undef, 2, 2)
π1 = Array{Float64}(undef, 2, 2, 2, 2)
X = Array{Float64}(undef, 2, 2)
α = Array{Float64}(undef, 2)
Xt1 = Array{Float64}(undef, 2, 2)
u̇_1t = Array{Float64}(undef, 2, 2)#here, u̇_1t is our updated guess for u̇_0t[:, :, 1], that is u̇_0t in the first period.

2×2 Matrix{Float64}:
 1.00971e-311  NaN
 3.5093e-320     0.0

2×2 Matrix{Float64}:
 1.00971e-311  NaN
 3.5093e-320     0.0

In [26]:
while !all(0.99 .*u̇_0t[:, :, 1] .<= u̇_1t .<= u̇_0t[:, :, 1] )
    println("NEXT ITERATION STARTS")
    println("u̇_0t at the beginning of the new iteration")
    println(u̇_0t)
    for n in 1:2
        for j in 1:2
            for i in 1:2
                for k in 1:2
                    denom = sum(μ_min1[n, j, m, h]*((u̇_0t[m, h, 2])^(β/ν)) for m in 1:2, h in 1:2)
                    println("denom")
                    println(denom)
                    numer = (μ_min1[n, j, i, k]*((u̇_0t[i, k, 2])^(β/ν)))
                    println("numer")
                    println(numer)
                    μ_0[n, j, i, k] = numer/denom
                end
            end
        end
    end
    println("μ_0")
    println(μ_0)
    for n in 1:2
        for j in 1:2
            L1[n, j] = sum(μ_0[i, k, n, j]*L0[i, k] for i in 1:2, k in 1:2)
        end
    end
    println("L1")
    println(L1)
    println("ẇn before the loop starts")
    println(ẇn)
    println("ẇ before the loop starts")
    println(ẇ)
    while !all(0.99*ẇ .<= ẇn .<= 1.01*ẇ)
        println("inner loop starts")
        for n in 1:2
            for i in 1:2
                for j in 1:2
                    Ṗ[n, j] = (sum(π0[n, j, i, j]*((ẇ[i, j]*κdot[n, j, i, j])^(-θ_list[j]))*((Ȧ[i, j])^θ_list[j]) for i in 1:2))^(-1/θ_list[j])   
                end
            end
        end
        println("Ṗ")
        println(Ṗ)
        for n in 1:2
            for i in 1:2
                for j in 1:2
                    π1[n, j, i, j] = π0[n, j, i, j]*(((ẇ[i, j]*κdot[n, j, i, j])/Ṗ[n, j])^(-θ_list[j]))*((Ȧ[i,j])^θ_list[j])
                end
            end
        end
        println("π1")
        println(π1)
        for i in 1:2
            for j in 1:2
                X[i, j] = w0[i, j]*L0[i, j]
            end
        end
        println("X")
        println(X)
        for i in 1:2
            α[i] = sum(X[k, i] for k in 1:2)/sum(X[j, l] for j in 1:2, l in 1:2)
        end
        println("α")
        println(α)
        for n in 1:2
            for j in 1:2
                Xt1[n, j] = α[j]*sum(ẇ[n, k]*L1[n, k]*w0[n, k] for k in 1:2)
            end
        end
        for n in 1:2
            for j in 1:2
                ẇn[n, j] = sum(π1[i, j, n, j]*Xt1[i, j] for i in 1:2)/(L1[n, j]*w0[n, j])
            end
        end
        println("ẇn at the end of inner loop")
        println(ẇn)
        copy!(ẇ, ẇn)
        println("ẇ reassigned")
        println(ẇ)
        println("inner loop ends")
    end
    println("ẇn after the loop ends")
    println(ẇn)
    println("ẇ after the loop ends")
    println(ẇ)
    for n in 1:2
        for j in 1:2
            u̇_1t[n, j] = ẇ[n, j]*(sum(μ_0[n, j, i, k]*(u̇_0t[i, k, 2])^(β/ν) for i in 1:2, k in 1:2))^(ν)
        end
    end
    println("u̇_0t before the loop")
    println(u̇_0t)
    println("u̇_1t")
    println(u̇_1t)
    # HERE FIRST STORE VALUE OF INITIAL UDOT FROM THE SECOND PERIOD INTO 
    u̇_0t[:, :, 1] .= u̇_1t
    println("u̇_0t updated and here it is after the loop")
    println(u̇_0t)
    println("ITERATION ENDS")
    println("------------------------------------------------------------------------------------------------")
    println("------------------------------------------------------------------------------------------------")
    println("------------------------------------------------------------------------------------------------")
end 

NEXT ITERATION STARTS
u̇_0t at the beginning of the new iteration
[1.007 0.997; 1.005 1.026;;; 1.006 1.003; 1.025 1.022]
denom
1.0081715562416365
numer
0.23905603454924615
denom
1.0081715562416365
numer
0.4077172405327442
denom
1.0081715562416365
numer
0.27282781803189665
denom
1.0081715562416365
numer
0.08857046312774952
denom
1.0128570010566391
numer
0.27868887208668286
denom
1.0128570010566391
numer
0.023709474852711247
denom
1.0128570010566391
numer
0.20991412270155926
denom
1.0128570010566391
numer
0.5005445314156859
denom
1.0101056782895528
numer
0.2567527641280955
denom
1.0101056782895528
numer
0.23255837052301573
denom
1.0101056782895528
numer
0.1467037904442507
denom
1.0101056782895528
numer
0.3740907531941908
denom
1.0113592080041027
numer
0.042313198526663785
denom
1.0113592080041027
numer
NEXT ITERATION STARTS
u̇_0t at the beginning of the new iteration
[1.007 0.997; 1.005 1.026;;; 1.006 1.003; 1.025 1.022]
denom
1.0081715562416365
numer
0.23905603454924615
denom
1.00817155